
# 📓 阶段 1 · 1.2 MiniLab —— Pearl 因果层级：关联 → 干预 → 反事实

本 Notebook 让你在 **15–30 分钟**内，亲手跑完三件事：  
1) **关联**：不动世界，只做相关回归；  
2) **干预**：用**背门调整**（IPTW）估计平均处理效应（ATE）；  
3) **反事实**：用一个**简化的 CATE/ITE** 模型，看看个体或亚组差异。

> 说明：为便于运行速度与可复现性，我们用**可控的模拟数据**：重症更可能被治疗（混杂），真实“治疗有益”。



## 0. 生成一份“有混杂”的模拟医学数据

- `age`、`sofa`：治疗前协变量（混杂）  
- `treat`：是否治疗（越重越容易被治疗，产生混杂）  
- `dead`：28 天死亡（真相：治疗降低死亡）


In [ ]:

import numpy as np, pandas as pd
import matplotlib.pyplot as plt

np.random.seed(2025)
n = 5000
age = np.random.normal(70, 10, n)                         # 年龄
sofa = np.clip(np.random.normal(5 + 0.1*(age-70), 2, n), 0, None)  # 病情严重度

# 治疗倾向：越重越可能接受治疗（混杂）
logit_t = -2 + 0.45*sofa + 0.01*(age-70)
p_treat = 1/(1+np.exp(-logit_t))
treat = (np.random.rand(n) < p_treat).astype(int)

# 结局：重症↑死亡↑，治疗可降低死亡（负向效应）
logit_y = -3 + 0.55*sofa + 0.02*(age-70) - 0.6*treat
p_dead = 1/(1+np.exp(-logit_y))
dead = (np.random.rand(n) < p_dead).astype(int)

df = pd.DataFrame({'age':age,'sofa':sofa,'treat':treat,'dead':dead})
df.head()



## 1. 关联（Association）：不动世界，只做相关回归

### 1.1 仅用 `treat` 回归 `dead`（**完全未调整**）
> 容易得到“治疗 ↔ 更高死亡”的**假象相关**（因为更重的病人更常被治疗）。


In [ ]:

import statsmodels.api as sm
import numpy as np

X0 = sm.add_constant(df[['treat']])
m0 = sm.Logit(df['dead'], X0).fit(disp=False)
print(m0.summary())
print("\n未调整模型：treat 的 OR ≈", np.exp(m0.params['treat']))



### 1.2 加入治疗前混杂 `age, sofa` 做**关联回归（仍非因果）**

> 这是“统计调整”的相关回归，不等同于 \( P(Y \mid do(X)) \)，但方向会更接近真实。


In [ ]:

X1 = sm.add_constant(df[['treat','age','sofa']])
m1 = sm.Logit(df['dead'], X1).fit(disp=False)
print(m1.summary())
print("\n加入混杂后：treat 的 OR ≈", np.exp(m1.params['treat']))



## 2. 干预（Intervention）：背门调整 → IPTW 估计 ATE

- 用 `age, sofa` 估计**倾向评分**（PS = 接受治疗的概率）；  
- 看**重叠**（PS 分布是否两组都有覆盖）；  
- 用 **IPTW** 计算加权后的两组死亡率差（ATE）。


In [ ]:

from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
import matplotlib.pyplot as plt

scaler = StandardScaler()
X_ps = scaler.fit_transform(df[['age','sofa']])
ps_model = LogisticRegression(max_iter=500).fit(X_ps, df['treat'])
ps = ps_model.predict_proba(X_ps)[:,1]
df['ps'] = ps

plt.figure()
plt.hist(df.loc[df.treat==1,'ps'], bins=30, alpha=0.6, label='Treatment')
plt.hist(df.loc[df.treat==0,'ps'], bins=30, alpha=0.6, label='Control')
plt.xlabel('Propensity Score'); plt.ylabel('Count'); plt.legend(); plt.title('PS Overlap')
plt.show()

# 稳定化权重（简化版）
w = np.where(df['treat']==1, 1/df['ps'], 1/(1-df['ps']))
df['w'] = w

ate_iptw = (df.loc[df.treat==1,'w']*df.loc[df.treat==1,'dead']).sum()/df.loc[df.treat==1,'w'].sum() \
         - (df.loc[df.treat==0,'w']*df.loc[df.treat==0,'dead']).sum()/df.loc[df.treat==0,'w'].sum()
print("IPTW 估计 ATE（>0=提高死亡，<0=降低死亡）：", round(ate_iptw, 4))

print("PS 分位数：", np.percentile(ps, [0,1,5,50,95,99,100]))



### 2.1 基线平衡（SMD）：加权前 vs 加权后

> 观察 SMD 是否向 0 靠拢（平衡性改善）。


In [ ]:

import numpy as np
import pandas as pd

def smd_cont(x_t, x_c):
    mu_t, mu_c = np.nanmean(x_t), np.nanmean(x_c)
    sd_pool = np.sqrt((np.nanvar(x_t, ddof=1)+np.nanvar(x_c, ddof=1))/2)
    return (mu_t - mu_c)/(sd_pool + 1e-12)

def smd_cont_w(x, t, w):
    # 加权均值差 / 未偏方差的近似合并（简化版示意）
    x_t, wt = x[t==1], w[t==1]
    x_c, wc = x[t==0], w[t==0]
    mu_t = np.sum(wt*x_t)/np.sum(wt)
    mu_c = np.sum(wc*x_c)/np.sum(wc)
    # 加权方差（近似）
    var_t = np.sum(wt*(x_t-mu_t)**2)/np.sum(wt)
    var_c = np.sum(wc*(x_c-mu_c)**2)/np.sum(wc)
    sd_pool = np.sqrt((var_t+var_c)/2 + 1e-12)
    return (mu_t - mu_c)/(sd_pool + 1e-12)

rows = []
for v in ['age','sofa']:
    pre = smd_cont(df.loc[df.treat==1,v], df.loc[df.treat==0,v])
    post = smd_cont_w(df[v].values, df['treat'].values, df['w'].values)
    rows.append((v, pre, post))

smd_tbl = pd.DataFrame(rows, columns=['variable','SMD_pre','SMD_post'])
smd_tbl['|pre|'] = smd_tbl['SMD_pre'].abs()
smd_tbl['|post|'] = smd_tbl['SMD_post'].abs()
smd_tbl.sort_values('|post|', ascending=False)



## 3. 反事实（Counterfactuals）：一个“入门级” CATE/ITE 演示

我们用 **T-learner** 思路：  
- 分别训练两套**结局模型**：`Y|X,T=1` 与 `Y|X,T=0`；  
- 对每个病人，估计两种世界的死亡概率，再取差值（`p1 - p0`，负值=治疗有益）。

> 这里只作直观演示，用的是 sklearn 分类器；严肃研究可用 EconML/因果森林/DR-learner 等。


In [ ]:

from sklearn.ensemble import GradientBoostingClassifier

features = ['age','sofa']
df_train_t = df[df.treat==1]
df_train_c = df[df.treat==0]

m1 = GradientBoostingClassifier().fit(df_train_t[features], df_train_t['dead'])
m0 = GradientBoostingClassifier().fit(df_train_c[features], df_train_c['dead'])

p1 = m1.predict_proba(df[features])[:,1]
p0 = m0.predict_proba(df[features])[:,1]
cate = p1 - p0  # 负值=治疗更好（更低死亡）
df['cate'] = cate

print("CATE 概览：中位数/25%/75%：", np.percentile(cate, [50,25,75]))

# 看看随 SOFA 的趋势（分箱后取均值）
bins = np.quantile(df['sofa'], [0, .2, .4, .6, .8, 1])
labels = [f"Q{i}" for i in range(1,6)]
df['sofa_bin'] = pd.cut(df['sofa'], bins=bins, labels=labels, include_lowest=True)
trend = df.groupby('sofa_bin')['cate'].mean().reset_index()
trend


In [ ]:

# 可视化：个体化效应分布与SOFA分位趋势
plt.figure()
plt.hist(df['cate'], bins=40, alpha=0.8)
plt.xlabel('Estimated ITE (p1 - p0)'); plt.ylabel('Count'); plt.title('ITE Distribution (T-learner)')
plt.show()

plt.figure()
plt.plot(trend['sofa_bin'], trend['cate'], marker='o')
plt.xlabel('SOFA quantile'); plt.ylabel('Mean ITE'); plt.title('ITE vs SOFA (coarse view)')
plt.show()



## 4. 小结（如何把三层用在你的研究里）

- **关联**：用于发现线索、做风险分层；**不要**直接下因果结论。  
- **干预**：先识别（背门/前门/IV/G方法），再估计（PSM/IPTW/DR/TMLE），**务必**检查平衡/重叠/敏感性。  
- **反事实**：在平均效应稳健后，再做 CATE/ITE；只在**有重叠**的区域解释个体化效应，并做稳健性检查。

> 你可以复制本 Notebook 的骨架到任何真实数据集：把 `age/sofa/treat/dead` 换成你的“治疗前协变量/治疗/结局”，其余流程不变。
